## 準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!git clone https://github.com/rs1004/research.git
%cd /content/research/ssd
%mkdir data
%cd data

In [ ]:
!unzip -q /content/drive/MyDrive/data/VOCdevkit.zip

In [ ]:
%cd /content/research/ssd

## 評価

In [ ]:
from torch.utils.data import Dataset, DataLoader
from utils.ssd_model import DataTransform
import json
from utils.ssd_model import SSD

In [ ]:
class EvalDataset(Dataset):
    def __init__(self, transform):
        self.image_list = self._get_image_list()
        self.transform = transform
    
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, idx):
        image_info = self.image_list[idx]
        id = image_info['id']
        file_name = image_info['file_name']
        image_file_path = './data/VOCdevkit/val/' + file_name
        img = cv2.imread(image_file_path)[..., ::-1]  # [高さ][幅][色BGR]
        height, width, channels = img.shape  # 画像のサイズを取得

        # 4. 前処理
        phase = "val"
        img_transformed, boxes, labels = self.transform(
            img, phase, "", "")  # アノテーションはないので、""にする
        img = torch.from_numpy(img_transformed).permute(2, 0, 1)
        
        return img, torch.tensor([id, height, width])
    
    def _get_image_list(self):
        with open('./data/VOCdevkit/instances_val.json') as f:
            image_list = json.load(f)['images']
        
        return image_list

In [ ]:
color_mean = (123, 117, 104)  # (BGR)の色の平均値
# color_std = (58, 57, 57)
color_std = None
input_size = 300 

dataset = EvalDataset(
    DataTransform(input_size, color_mean, color_std)
)

dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=False
)

In [ ]:
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="inference", cfg=ssd_cfg)

## 評価実行

In [ ]:
from pathlib import Path
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm
from utils.ssd_model import Detect

In [ ]:
net.eval()
net.cuda()

detect = Detect(conf_thresh=0.1)

for weights_path in Path('/content/drive/MyDrive/weights/std').glob('ssd300*.pth'):
    net_weights = torch.load(weights_path, map_location='cpu')
    net.load_state_dict(net_weights)
    
    ret = []
    print(weights_path.as_posix())
    for images, metas in dataloader:
        images = images.cuda()

        with torch.no_grad():
            outputs = net(images)
            detections = detect(outputs[0], outputs[1], outputs[2].cuda())
            for detection, meta in zip(detections, metas):
                id, height, width = meta
                for class_id, row in enumerate(detection):
                    for score, xmin, ymin, xmax, ymax in row:
                        if score >= 0.1:
                            ret.append({
                                'image_id': id.item(),
                                'category_id': class_id,
                                'bbox': [
                                    xmin.item() * width.item(),
                                    ymin.item() * height.item(),
                                    (xmax - xmin).item() * width.item(),
                                    (ymax - ymin).item() * height.item()
                                ],
                                'score': score.item()
                            })
    cocoGt = COCO('./data/VOCdevkit/instances_val.json')
    cocoDt = cocoGt.loadRes(ret)
    cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()